In [3]:
import sys 
import subprocess
import os
import json
import geopandas as gpd
import pandas as pd

In [4]:
resolution = "50"

In [5]:
folder_path = f"/home/data/national_fema_floodmap/rasters/{resolution}m"
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

In [6]:
raster100y =  f"{folder_path}/national_fema_raster_100y_{resolution}m.tiff"
raster500y =  f"{folder_path}/national_fema_raster_500y_{resolution}m.tiff"
raster_combined = f"{folder_path}/national_fema_raster_combined_{resolution}m.tiff"

In [13]:
# Define the command as a list of arguments
command = [
    "gdal_rasterize",
    "-tr", resolution, resolution,
    "-sql", "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '1% Annual Chance Flood Hazard'",
    "-burn", "1",
    "-ot", "Byte",
    "-co", "COMPRESS=LZW",
    "/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb",
    raster100y
]


# Execute the command
subprocess.run(command, check=True)

ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
Warning 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined


0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdal_rasterize', '-tr', '30', '30', '-sql', "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '1% Annual Chance Flood Hazard'", '-burn', '1', '-ot', 'Byte', '-co', 'COMPRESS=LZW', '/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_100y_30m.tiff'], returncode=0)

In [14]:
import subprocess

# Define the command as a list of arguments
command = [
    "gdal_rasterize",
    "-tr", resolution, resolution,
    "-sql", "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '0.2% Annual Chance Flood Hazard'",
    "-burn", "1",
    "-ot", "Byte",
    "-co", "COMPRESS=LZW",
    "/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb",
    raster500y
]


# Execute the command
subprocess.run(command, check=True)

ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
Warning 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined


0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdal_rasterize', '-tr', '30', '30', '-sql', "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '0.2% Annual Chance Flood Hazard'", '-burn', '1', '-ot', 'Byte', '-co', 'COMPRESS=LZW', '/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_500y_30m.tiff'], returncode=0)

In [19]:
# Construct the command using sys.executable to ensure the correct Python interpreter is used
command = [
    sys.executable, "-m", "osgeo_utils.gdal_calc",
    "-A", raster100y,
    "-B", raster500y,
    "--outfile=" + raster_combined,
    "--calc=\"(A>0)*1 + (B>0)*(A<=0)*2\"",  # Conditional calculation
    "--NoDataValue=0",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    "--co", "COMPRESS=LZW",
    "--overwrite"
]# Execute the command
subprocess.run(command, check=True)

/home/canyon/miniconda3/envs/myenv/lib/python3.9/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['/home/canyon/miniconda3/envs/myenv/bin/python', '-m', 'osgeo_utils.gdal_calc', '-A', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_100y_30m.tiff', '-B', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_500y_30m.tiff', '--outfile=/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_combined_30m.tiff', '--calc="(A>0)*1 + (B>0)*(A<=0)*2"', '--NoDataValue=0', '--config', 'CHECK_DISK_FREE_SPACE', 'FALSE', '--co', 'COMPRESS=LZW', '--overwrite'], returncode=0)

In [7]:
import json
with open("/home/canyon/TOP-Sprint/analysis/src/config.json") as f:
        config = json.load(f)

In [8]:
def create_national_boundary(config):
    gdfs = []
    cities_base_path = f"{config['base_path']}/cities/"
    for city in next(os.walk(cities_base_path))[1]:
        try:
            print(f"Reading data for {city}")
            tracts = gpd.read_file(f"{cities_base_path}/{city}/census/geo/tracts.geojson")
            tracts['to_merge'] = 1
    
            gdf_extent = tracts.dissolve(by='to_merge')
            gdfs.append(gdf_extent)
        except:
            print(f"No geography for {city}")
            continue
    national_gdf = pd.concat(gdfs)

    national_gdf = national_gdf.reset_index().dissolve(by='to_merge')
    return national_gdf

In [9]:
out = create_national_boundary(config)

Reading data for C1382
Reading data for C1074
Reading data for C3806
Reading data for C3108
Reading data for C4014
Reading data for C4186
Reading data for C4090
Reading data for C3538
Reading data for C4174
Reading data for C4194
Reading data for C2342
Reading data for C1974
Reading data for C2554
Reading data for C1486
Reading data for C4790
Reading data for C3310
Reading data for C4530
Reading data for C3674
Reading data for C2726
Reading data for C1206
Reading data for C1978
Reading data for C1698
Reading data for C2690
Reading data for C3114
Reading data for C1446
Reading data for C4934
Reading data for C1258
Reading data for C1982
Reading data for C3346
Reading data for C4118
Reading data for C2814
Reading data for C1674
Reading data for C3958
Reading data for C3654
Reading data for C2982
Reading data for C3562
Reading data for C4038
Reading data for C1538
Reading data for C1746
Reading data for C1714
Reading data for C1814
Reading data for C3642
Reading data for C3890
Reading dat

In [10]:
boundary_path = "/home/data/national_fema_floodmap/national_boundaries.geojson" 
out.to_file(boundary_path)

In [11]:
clipped_raster_path = f"{folder_path}/national_fema_raster_combined_clipped.tiff"

In [12]:
raster_combined

'/home/data/national_fema_floodmap/rasters/50m/national_fema_raster_combined_50m.tiff'

In [14]:
command = [
    "gdalwarp", "-cutline",
    boundary_path, "-crop_to_cutline",
    "-dstalpha", "-co", "COMPRESS=LZW", "-co", "PREDICTOR=2", "-co", "TILED=YES",
    raster_combined,
    clipped_raster_path
]

# Execute the command
subprocess.run(command, check=True)

Creating output file that is 118577P x 65279L.
Processing /home/data/national_fema_floodmap/rasters/50m/national_fema_raster_combined_50m.tiff [1/1] : 0Using internal nodata values (e.g. 0) for image /home/data/national_fema_floodmap/rasters/50m/national_fema_raster_combined_50m.tiff.
...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdalwarp', '-cutline', '/home/data/national_fema_floodmap/national_boundaries.geojson', '-crop_to_cutline', '-dstalpha', '-co', 'COMPRESS=LZW', '-co', 'PREDICTOR=2', '-co', 'TILED=YES', '/home/data/national_fema_floodmap/rasters/50m/national_fema_raster_combined_50m.tiff', '/home/data/national_fema_floodmap/rasters/50m/national_fema_raster_combined_clipped.tiff'], returncode=0)